## Récupérer les coordonnées des centres de distribution

In [12]:
import pandas as pd

df = pd.read_excel("data/centres.xlsx")

In [13]:
names = df["Centre"].tolist()
addresses = ["Restos du Coeur " + name for name in names]
addresses[11] = "Rue du 19 Mars 1962 Fronton" # Le centre de Fronton n'est pas sur Maps

In [14]:
import requests

api_key = open("api_key", "r").read()

In [15]:
latitudes = []
longitudes = []

for address in addresses:
    response = requests.get(
        f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    )
    data = response.json()

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        latitudes.append(location["lat"])
        longitudes.append(location["lng"])
    else:
        print(f"Geocoding failed for address {address}. Status:", data["status"])
        

In [16]:
# Add the latitudes as new columns into the original excel file
df["Latitude"] = latitudes
df["Longitude"] = longitudes

In [17]:
df.to_excel("data/centres.xlsx", index=False)

## Créer la matrice de distances

In [18]:
import googlemaps

gmaps = googlemaps.Client(key=api_key)

In [19]:
from tqdm import tqdm

matrix = []

for i, a in tqdm(enumerate(addresses), total=len(addresses)):
    matrix.append([])
    for j, b in enumerate(addresses):
        res = gmaps.distance_matrix(a, b)
        matrix[i].append(res["rows"][0]["elements"][0]["distance"]["value"])

100%|██████████| 29/29 [00:54<00:00,  1.88s/it]


In [20]:
mat = pd.DataFrame(matrix, columns=names, index=names)
mat.to_excel("data/matrix.xlsx")